# Original UNet

In [ ]:
print("Hello")

Hello


In [ ]:
!unzip CWD-Github.zip

Archive:  CWD-Github.zip
   creating: CWD-Github/
   creating: CWD-Github/train/
   creating: CWD-Github/train/images/
  inflating: CWD-Github/train/images/346_jpg.rf.f49547e8339ac156a2280821a00ca6c9.jpg  
  inflating: CWD-Github/train/images/937_jpg.rf.bb552151d5941ed19d6c86415e168c4b.jpg  
  inflating: CWD-Github/train/images/1240_jpg.rf.64ff0f765e32e144cd007b4813c7e7dc.jpg  
  inflating: CWD-Github/train/images/891_jpg.rf.3ee8086efb9cc27df3874656217957ef.jpg  
  inflating: CWD-Github/train/images/1498_jpg.rf.2ada5fc7d17908f870925ac3d946143e.jpg  
  inflating: CWD-Github/train/images/975_jpg.rf.82fe20ef65de95ff02b163bef07502c1.jpg  
  inflating: CWD-Github/train/images/1569_jpg.rf.336d6a9b982e1bb27864e6210748370a.jpg  
  inflating: CWD-Github/train/images/1548_jpg.rf.623b857c66d0e60c22c41d9186eac0dc.jpg  
  inflating: CWD-Github/train/images/1610_jpg.rf.1ae2a46f613fd1b8b85ea51761a6fcd8.jpg  
  inflating: CWD-Github/train/images/255_jpg.rf.963e783afd9eba2547d1cf7d45d6b6f1.jpg  
  infl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bridge
        self.bridge = self.conv_block(512, 1024)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        # Final output layer
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # ---------------- Encoder ----------------
        enc1 = self.enc1(x)                      # (B,64,H,W)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))   # (B,128,H/2,W/2)
        enc3 = self.enc3(F.max_pool2d(enc2, 2))   # (B,256,H/4,W/4)
        enc4 = self.enc4(F.max_pool2d(enc3, 2))   # (B,512,H/8,W/8)

        # ---------------- Bridge ----------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B,1024,H/16,W/16)

        # ---------------- Decoder ----------------
        # Up4
        dec4 = self.up4(bridge)                    # (B,512,H/8,W/8)
        dec4 = torch.cat([enc4, dec4], dim=1)      # (B,1024,H/8,W/8)
        dec4 = self.dec4(dec4)                     # (B,512,H/8,W/8)

        # Up3
        dec3 = self.up3(dec4)                      # (B,256,H/4,W/4)
        dec3 = torch.cat([enc3, dec3], dim=1)      # (B,512,H/4,W/4)
        dec3 = self.dec3(dec3)                     # (B,256,H/4,W/4)

        # Up2
        dec2 = self.up2(dec3)                      # (B,128,H/2,W/2)
        dec2 = torch.cat([enc2, dec2], dim=1)      # (B,256,H/2,W/2)
        dec2 = self.dec2(dec2)                     # (B,128,H/2,W/2)

        # Up1
        dec1 = self.up1(dec2)                      # (B,64,H,W)
        dec1 = torch.cat([enc1, dec1], dim=1)      # (B,128,H,W)
        dec1 = self.dec1(dec1)                     # (B,64,H,W)

        # Output
        out = self.out(dec1)                       # (B,out_channels,H,W)
        return out

# UNet with Batch Normalization

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet_BN(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_BN, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bridge
        self.bridge = self.conv_block(512, 1024)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        # Final output layer
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        """(Conv -> BN -> ReLU) × 2 block."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # ---------------- Encoder ----------------
        enc1 = self.enc1(x)                      # (B,64,H,W)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))   # (B,128,H/2,W/2)
        enc3 = self.enc3(F.max_pool2d(enc2, 2))   # (B,256,H/4,W/4)
        enc4 = self.enc4(F.max_pool2d(enc3, 2))   # (B,512,H/8,W/8)

        # ---------------- Bridge ----------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B,1024,H/16,W/16)

        # ---------------- Decoder ----------------
        # Up4
        dec4 = self.up4(bridge)                    # (B,512,H/8,W/8)
        dec4 = torch.cat([enc4, dec4], dim=1)      # (B,1024,H/8,W/8)
        dec4 = self.dec4(dec4)                     # (B,512,H/8,W/8)

        # Up3
        dec3 = self.up3(dec4)                      # (B,256,H/4,W/4)
        dec3 = torch.cat([enc3, dec3], dim=1)      # (B,512,H/4,W/4)
        dec3 = self.dec3(dec3)                     # (B,256,H/4,W/4)

        # Up2
        dec2 = self.up2(dec3)                      # (B,128,H/2,W/2)
        dec2 = torch.cat([enc2, dec2], dim=1)      # (B,256,H/2,W/2)
        dec2 = self.dec2(dec2)                     # (B,128,H/2,W/2)

        # Up1
        dec1 = self.up1(dec2)                      # (B,64,H,W)
        dec1 = torch.cat([enc1, dec1], dim=1)      # (B,128,H,W)
        dec1 = self.dec1(dec1)                     # (B,64,H,W)

        # Output
        out = self.out(dec1)                       # (B,out_channels,H,W)
        return out

# UNet With Spatial Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Channel-wise average pooling
        avg_out = torch.mean(x, dim=1, keepdim=True)
        # Channel-wise max pooling
        max_out, _ = torch.max(x, dim=1, keepdim=True)

        # Concatenate along channel dimension
        out = torch.cat([avg_out, max_out], dim=1)

        # Convolve, then apply sigmoid -> attention mask
        attn = self.conv1(out)
        attn = self.sigmoid(attn)

        # Multiply the attention mask with the input
        return x * attn

class UNet_SA(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_SA, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bridge
        self.bridge = self.conv_block(512, 1024)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)

        # Final output layer
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

        # Spatial Attention modules
        self.sa1 = SpatialAttention()
        self.sa2 = SpatialAttention()
        self.sa3 = SpatialAttention()
        self.sa4 = SpatialAttention()
        self.sa_bridge = SpatialAttention()

    def conv_block(self, in_channels, out_channels):
        """
        A basic (Conv -> ReLU -> Conv -> ReLU) block,
        no Batch Normalization layers.
        """
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # ------------------ Encoder ------------------
        enc1 = self.enc1(x)            # (B,64,H,W)
        enc1 = self.sa1(enc1)          # Spatial attention

        enc2 = self.enc2(F.max_pool2d(enc1, 2))  # (B,128,H/2,W/2)
        enc2 = self.sa2(enc2)

        enc3 = self.enc3(F.max_pool2d(enc2, 2))  # (B,256,H/4,W/4)
        enc3 = self.sa3(enc3)

        enc4 = self.enc4(F.max_pool2d(enc3, 2))  # (B,512,H/8,W/8)
        enc4 = self.sa4(enc4)

        # ------------------ Bridge -------------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B,1024,H/16,W/16)
        bridge = self.sa_bridge(bridge)

        # ------------------ Decoder ------------------
        # Up4
        dec4 = self.up4(bridge)                 # (B,512,H/8,W/8)
        dec4 = torch.cat([enc4, dec4], dim=1)   # (B,1024,H/8,W/8)
        dec4 = self.dec4(dec4)                  # (B,512,H/8,W/8)

        # Up3
        dec3 = self.up3(dec4)                   # (B,256,H/4,W/4)
        dec3 = torch.cat([enc3, dec3], dim=1)   # (B,512,H/4,W/4)
        dec3 = self.dec3(dec3)                  # (B,256,H/4,W/4)

        # Up2
        dec2 = self.up2(dec3)                   # (B,128,H/2,W/2)
        dec2 = torch.cat([enc2, dec2], dim=1)   # (B,256,H/2,W/2)
        dec2 = self.dec2(dec2)                  # (B,128,H/2,W/2)

        # Up1
        dec1 = self.up1(dec2)                   # (B,64,H,W)
        dec1 = torch.cat([enc1, dec1], dim=1)   # (B,128,H,W)
        dec1 = self.dec1(dec1)                  # (B,64,H,W)

        # Final conv
        out = self.out(dec1)                    # (B,out_channels,H,W)
        return out

# UNet with dense skip connections

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet_DSC(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_DSC, self).__init__()
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.bridge = self.conv_block(512, 1024)
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(512+512+256+128+64, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(256+256+128+64, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(128+128+64, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(64+64, 64)
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)
    def conv_block(self, in_channels, out_channels):
        """
        Two successive (Conv -> ReLU) layers (no batch normalization).
        Adjust if you want BN or other features.
        """
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        B,C,H,W = x.shape
        enc1 = self.enc1(x)
        e2_in= F.max_pool2d(enc1, 2)
        enc2 = self.enc2(e2_in)
        e3_in= F.max_pool2d(enc2, 2)
        enc3 = self.enc3(e3_in)
        e4_in= F.max_pool2d(enc3, 2)
        enc4 = self.enc4(e4_in)
        b_in = F.max_pool2d(enc4, 2)
        bridge = self.bridge(b_in)
        dec4_up = self.up4(bridge)
        enc1_8 = F.interpolate(enc1, scale_factor=1/8, mode='bilinear', align_corners=False)
        enc2_8 = F.interpolate(enc2, scale_factor=1/4, mode='bilinear', align_corners=False)
        enc3_8 = F.interpolate(enc3, scale_factor=1/2, mode='bilinear', align_corners=False)
        dec4_in = torch.cat([dec4_up, enc4, enc3_8, enc2_8, enc1_8], dim=1)
        dec4 = self.dec4(dec4_in)
        dec3_up = self.up3(dec4)
        enc1_4 = F.interpolate(enc1, scale_factor=1/4, mode='bilinear', align_corners=False)
        enc2_4 = F.interpolate(enc2, scale_factor=1/2, mode='bilinear', align_corners=False)
        dec3_in= torch.cat([dec3_up, enc3, enc2_4, enc1_4], dim=1)
        dec3   = self.dec3(dec3_in)
        dec2_up= self.up2(dec3)
        enc1_2 = F.interpolate(enc1, scale_factor=1/2, mode='bilinear', align_corners=False)
        dec2_in= torch.cat([dec2_up, enc2, enc1_2], dim=1)
        dec2   = self.dec2(dec2_in)
        dec1_up= self.up1(dec2)
        dec1_in= torch.cat([dec1_up, enc1], dim=1)
        dec1   = self.dec1(dec1_in)
        out = self.out(dec1)
        return out


# UNet With Spatial Attention + Batch Normalization + Dense Skip Connections

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size,
                              padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, H, W)
        # Channel-wise avg
        avg_out = torch.mean(x, dim=1, keepdim=True)  # (B,1,H,W)
        # Channel-wise max
        max_out, _ = torch.max(x, dim=1, keepdim=True)# (B,1,H,W)
        # cat => (B,2,H,W)
        attn_in = torch.cat([avg_out, max_out], dim=1)
        attn    = self.conv(attn_in)     # (B,1,H,W)
        attn    = self.sigmoid(attn)
        return x * attn


class UNet_SA_BN_DSC(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_SA_BN_DSC, self).__init__()

        # ---- Encoder blocks (with BN + ReLU) ----
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64,   128)
        self.enc3 = self.conv_block(128,  256)
        self.enc4 = self.conv_block(256,  512)

        # ---- Bridge block ----
        self.bridge = self.conv_block(512, 1024)

        # ---- Spatial Attention modules for each stage ----
        self.sa1      = SpatialAttention()
        self.sa2      = SpatialAttention()
        self.sa3      = SpatialAttention()
        self.sa4      = SpatialAttention()
        self.sa_bridge= SpatialAttention()

        # ---- Decoder blocks (with BN + ReLU) + dense skip logic ----
        # dec4: up from 1024 -> 512, merges enc4 + upsampled (enc3, enc2, enc1)
        # total cat => 512 + 512 + 256 + 128 + 64 = 1472
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4= self.conv_block(512+512+256+128+64, 512)

        # dec3: up from 512 -> 256, merges enc3 + upsampled (enc2, enc1)
        # total cat => 256 + 256 + 128 + 64 = 704
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3= self.conv_block(256+256+128+64, 256)

        # dec2: up from 256 -> 128, merges enc2 + upsampled (enc1)
        # total cat => 128 + 128 + 64 = 320
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2= self.conv_block(128+128+64, 128)

        # dec1: up from 128 -> 64, merges enc1
        # total cat => 64 + 64 = 128
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1= self.conv_block(64+64, 64)

        # ---- Final output 1×1 conv ----
        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

    # --------------------------
    #  2× (Conv -> BN -> ReLU)
    # --------------------------
    def conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        B,C,H,W = x.shape

        # =============== Encoder ===============
        # enc1 => (B,64,H,W)
        enc1 = self.enc1(x)
        enc1 = self.sa1(enc1)

        e2_in= F.max_pool2d(enc1, 2)               # (H/2,W/2)
        enc2 = self.enc2(e2_in)                   # (B,128,H/2,W/2)
        enc2 = self.sa2(enc2)

        e3_in= F.max_pool2d(enc2, 2)              # (H/4,W/4)
        enc3 = self.enc3(e3_in)                   # (B,256,H/4,W/4)
        enc3 = self.sa3(enc3)

        e4_in= F.max_pool2d(enc3, 2)              # (H/8,W/8)
        enc4 = self.enc4(e4_in)                   # (B,512,H/8,W/8)
        enc4 = self.sa4(enc4)

        # =============== Bridge ===============
        b_in= F.max_pool2d(enc4, 2)               # (H/16,W/16)
        bridge = self.bridge(b_in)                # (B,1024,H/16,W/16)
        bridge = self.sa_bridge(bridge)

        # =============== Decoder ===============
        # dec4 => up from H/16->H/8
        dec4_up= self.up4(bridge)                 # (B,512,H/8,W/8)

        # We want enc4(H/8) + upsampled enc3(H/4->H/8) + enc2(H/2->H/8) + enc1(H->H/8)
        enc3_8= F.interpolate(enc3, scale_factor=1/2, mode='bilinear', align_corners=False)
        enc2_8= F.interpolate(enc2, scale_factor=1/4, mode='bilinear', align_corners=False)
        enc1_8= F.interpolate(enc1, scale_factor=1/8, mode='bilinear', align_corners=False)
        dec4_in= torch.cat([dec4_up, enc4, enc3_8, enc2_8, enc1_8], dim=1)
        dec4   = self.dec4(dec4_in)               # (B,512,H/8,W/8)

        # dec3 => up from H/8->H/4
        dec3_up= self.up3(dec4)                   # (B,256,H/4,W/4)
        # cat enc3(H/4), enc2(H/2->H/4), enc1(H->H/4)
        enc2_4= F.interpolate(enc2, scale_factor=1/2, mode='bilinear', align_corners=False)
        enc1_4= F.interpolate(enc1, scale_factor=1/4, mode='bilinear', align_corners=False)
        dec3_in= torch.cat([dec3_up, enc3, enc2_4, enc1_4], dim=1)
        dec3   = self.dec3(dec3_in)               # (B,256,H/4,W/4)

        # dec2 => up from H/4->H/2
        dec2_up= self.up2(dec3)                   # (B,128,H/2,W/2)
        # cat enc2(H/2), enc1(H->H/2)
        enc1_2= F.interpolate(enc1, scale_factor=1/2, mode='bilinear', align_corners=False)
        dec2_in= torch.cat([dec2_up, enc2, enc1_2], dim=1)
        dec2   = self.dec2(dec2_in)               # (B,128,H/2,W/2)

        # dec1 => up from H/2->H
        dec1_up= self.up1(dec2)                   # (B,64,H,W)
        # cat enc1(H)
        dec1_in= torch.cat([dec1_up, enc1], dim=1)# (B,128,H,W)
        dec1   = self.dec1(dec1_in)               # (B,64,H,W)

        out= self.out(dec1)                       # (B,out_channels,H,W)
        return out


In [ ]:
# Cell: Additional Advanced Models for Crop-Weed Detection

import torch
import torch.nn as nn
import torch.nn.functional as F

# ==============================================================================
# 1. DeepLabV3+ (Highly effective for agricultural segmentation)
# ==============================================================================
class ASPP(nn.Module):
    """Atrous Spatial Pyramid Pooling"""
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()

        # Different dilation rates
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=6, dilation=6, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=12, dilation=12, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=18, dilation=18, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        # Global average pooling
        self.global_avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

        self.conv_out = nn.Sequential(
            nn.Conv2d(out_channels * 5, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        size = x.shape[2:]

        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=size, mode='bilinear', align_corners=False)

        x = torch.cat([x1, x2, x3, x4, x5], dim=1)
        x = self.conv_out(x)
        return x


class DeepLabV3Plus(nn.Module):
    """DeepLabV3+ - Excellent for crop-weed segmentation"""
    def __init__(self, in_channels=3, out_channels=3):
        super(DeepLabV3Plus, self).__init__()

        # Encoder
        self.enc1 = self._make_layer(in_channels, 64)
        self.enc2 = self._make_layer(64, 128)
        self.enc3 = self._make_layer(128, 256)
        self.enc4 = self._make_layer(256, 512)

        # ASPP Module
        self.aspp = ASPP(512, 256)

        # Low-level feature processing
        self.low_level_conv = nn.Sequential(
            nn.Conv2d(128, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(304, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1)
        )

        self.final = nn.Conv2d(256, out_channels, 1)

    def _make_layer(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        size = x.shape[2:]

        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        enc4 = self.enc4(F.max_pool2d(enc3, 2))

        # ASPP
        aspp_out = self.aspp(F.max_pool2d(enc4, 2))
        aspp_out = F.interpolate(aspp_out, size=enc2.shape[2:], mode='bilinear', align_corners=False)

        # Low-level features
        low_level = self.low_level_conv(enc2)

        # Concatenate
        decoder_in = torch.cat([aspp_out, low_level], dim=1)

        # Decoder
        out = self.decoder(decoder_in)
        out = F.interpolate(out, size=size, mode='bilinear', align_corners=False)
        out = self.final(out)

        return out


# ==============================================================================
# 2. PSPNet (Pyramid Scene Parsing Network)
# ==============================================================================
class PyramidPooling(nn.Module):
    """Pyramid Pooling Module"""
    def __init__(self, in_channels, out_channels=512):
        super(PyramidPooling, self).__init__()

        self.pool1 = nn.AdaptiveAvgPool2d(1)
        self.pool2 = nn.AdaptiveAvgPool2d(2)
        self.pool3 = nn.AdaptiveAvgPool2d(3)
        self.pool4 = nn.AdaptiveAvgPool2d(6)

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 4, 1, bias=False),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 4, 1, bias=False),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 4, 1, bias=False),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 4, 1, bias=False),
            nn.BatchNorm2d(out_channels // 4),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        size = x.shape[2:]

        feat1 = F.interpolate(self.conv1(self.pool1(x)), size=size, mode='bilinear', align_corners=False)
        feat2 = F.interpolate(self.conv2(self.pool2(x)), size=size, mode='bilinear', align_corners=False)
        feat3 = F.interpolate(self.conv3(self.pool3(x)), size=size, mode='bilinear', align_corners=False)
        feat4 = F.interpolate(self.conv4(self.pool4(x)), size=size, mode='bilinear', align_corners=False)

        return torch.cat([x, feat1, feat2, feat3, feat4], dim=1)


class PSPNet(nn.Module):
    """PSPNet - Great for multi-scale feature extraction"""
    def __init__(self, in_channels=3, out_channels=3):
        super(PSPNet, self).__init__()

        # Encoder
        self.enc1 = self._make_layer(in_channels, 64)
        self.enc2 = self._make_layer(64, 128)
        self.enc3 = self._make_layer(128, 256)
        self.enc4 = self._make_layer(256, 512)

        # Pyramid Pooling
        self.ppm = PyramidPooling(512, 512)

        # Final classifier
        self.final = nn.Sequential(
            nn.Conv2d(1024, 512, 3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Conv2d(512, out_channels, 1)
        )

    def _make_layer(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        size = x.shape[2:]

        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        enc4 = self.enc4(F.max_pool2d(enc3, 2))

        # Pyramid pooling
        ppm_out = self.ppm(enc4)

        # Final
        out = self.final(ppm_out)
        out = F.interpolate(out, size=size, mode='bilinear', align_corners=False)

        return out


# ==============================================================================
# 3. FPN (Feature Pyramid Network) - Lightweight and effective
# ==============================================================================
class FPN(nn.Module):
    """Feature Pyramid Network - Good for small objects (weeds)"""
    def __init__(self, in_channels=3, out_channels=3):
        super(FPN, self).__init__()

        # Bottom-up pathway (encoder)
        self.enc1 = self._make_layer(in_channels, 64)
        self.enc2 = self._make_layer(64, 128)
        self.enc3 = self._make_layer(128, 256)
        self.enc4 = self._make_layer(256, 512)

        # Lateral connections
        self.lat4 = nn.Conv2d(512, 256, 1)
        self.lat3 = nn.Conv2d(256, 256, 1)
        self.lat2 = nn.Conv2d(128, 256, 1)
        self.lat1 = nn.Conv2d(64, 256, 1)

        # Top-down pathway
        self.smooth4 = nn.Conv2d(256, 256, 3, padding=1)
        self.smooth3 = nn.Conv2d(256, 256, 3, padding=1)
        self.smooth2 = nn.Conv2d(256, 256, 3, padding=1)
        self.smooth1 = nn.Conv2d(256, 256, 3, padding=1)

        # Final segmentation head
        self.seg_head = nn.Sequential(
            nn.Conv2d(256 * 4, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, out_channels, 1)
        )

    def _make_layer(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        size = x.shape[2:]

        # Bottom-up
        c1 = self.enc1(x)
        c2 = self.enc2(F.max_pool2d(c1, 2))
        c3 = self.enc3(F.max_pool2d(c2, 2))
        c4 = self.enc4(F.max_pool2d(c3, 2))

        # Top-down
        p4 = self.lat4(c4)
        p3 = self.lat3(c3) + F.interpolate(p4, size=c3.shape[2:], mode='nearest')
        p2 = self.lat2(c2) + F.interpolate(p3, size=c2.shape[2:], mode='nearest')
        p1 = self.lat1(c1) + F.interpolate(p2, size=c1.shape[2:], mode='nearest')

        # Smooth
        p4 = self.smooth4(p4)
        p3 = self.smooth3(p3)
        p2 = self.smooth2(p2)
        p1 = self.smooth1(p1)

        # Upsample all to original size
        p4 = F.interpolate(p4, size=size, mode='bilinear', align_corners=False)
        p3 = F.interpolate(p3, size=size, mode='bilinear', align_corners=False)
        p2 = F.interpolate(p2, size=size, mode='bilinear', align_corners=False)

        # Concatenate and predict
        out = torch.cat([p1, p2, p3, p4], dim=1)
        out = self.seg_head(out)

        return out


# ==============================================================================
# 4. U-Net++ (Nested U-Net) - Advanced skip connections
# ==============================================================================
class UNetPlusPlus(nn.Module):
    """U-Net++ with nested skip pathways"""
    def __init__(self, in_channels=3, out_channels=3):
        super(UNetPlusPlus, self).__init__()

        # Encoder
        self.conv0_0 = self._conv_block(in_channels, 64)
        self.conv1_0 = self._conv_block(64, 128)
        self.conv2_0 = self._conv_block(128, 256)
        self.conv3_0 = self._conv_block(256, 512)
        self.conv4_0 = self._conv_block(512, 1024)

        # Nested skip connections
        self.conv0_1 = self._conv_block(64 + 128, 64)
        self.conv1_1 = self._conv_block(128 + 256, 128)
        self.conv2_1 = self._conv_block(256 + 512, 256)
        self.conv3_1 = self._conv_block(512 + 1024, 512)

        self.conv0_2 = self._conv_block(64 * 2 + 128, 64)
        self.conv1_2 = self._conv_block(128 * 2 + 256, 128)
        self.conv2_2 = self._conv_block(256 * 2 + 512, 256)

        self.conv0_3 = self._conv_block(64 * 3 + 128, 64)
        self.conv1_3 = self._conv_block(128 * 3 + 256, 128)

        self.conv0_4 = self._conv_block(64 * 4 + 128, 64)

        # Upsampling layers
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # Final output
        self.final = nn.Conv2d(64, out_channels, 1)

    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Column 0
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(F.max_pool2d(x0_0, 2))
        x2_0 = self.conv2_0(F.max_pool2d(x1_0, 2))
        x3_0 = self.conv3_0(F.max_pool2d(x2_0, 2))
        x4_0 = self.conv4_0(F.max_pool2d(x3_0, 2))

        # Column 1
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))

        # Column 2
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))

        # Column 3
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))

        # Column 4
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        output = self.final(x0_4)
        return output


# ==============================================================================
# 5. Attention U-Net - Channel and Spatial Attention
# ==============================================================================
class AttentionGate(nn.Module):
    """Attention Gate Module"""
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()

        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, 1, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, 1, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, 1, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class AttentionUNet(nn.Module):
    """Attention U-Net for better feature selection"""
    def __init__(self, in_channels=3, out_channels=3):
        super(AttentionUNet, self).__init__()

        # Encoder
        self.enc1 = self._conv_block(in_channels, 64)
        self.enc2 = self._conv_block(64, 128)
        self.enc3 = self._conv_block(128, 256)
        self.enc4 = self._conv_block(256, 512)

        # Bridge
        self.bridge = self._conv_block(512, 1024)

        # Attention gates
        self.att4 = AttentionGate(F_g=1024, F_l=512, F_int=512)
        self.att3 = AttentionGate(F_g=512, F_l=256, F_int=256)
        self.att2 = AttentionGate(F_g=256, F_l=128, F_int=128)
        self.att1 = AttentionGate(F_g=128, F_l=64, F_int=64)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = self._conv_block(1024, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = self._conv_block(512, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = self._conv_block(256, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = self._conv_block(128, 64)

        self.out = nn.Conv2d(64, out_channels, 1)

    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        enc4 = self.enc4(F.max_pool2d(enc3, 2))

        # Bridge
        bridge = self.bridge(F.max_pool2d(enc4, 2))

        # Decoder with attention
        dec4 = self.up4(bridge)
        enc4_att = self.att4(g=dec4, x=enc4)
        dec4 = torch.cat([enc4_att, dec4], dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.up3(dec4)
        enc3_att = self.att3(g=dec3, x=enc3)
        dec3 = torch.cat([enc3_att, dec3], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.up2(dec3)
        enc2_att = self.att2(g=dec2, x=enc2)
        dec2 = torch.cat([enc2_att, dec2], dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.up1(dec2)
        enc1_att = self.att1(g=dec1, x=enc1)
        dec1 = torch.cat([enc1_att, dec1], dim=1)
        dec1 = self.dec1(dec1)

        return self.out(dec1)

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------------------------
# Configuration - OPTIMIZED FOR MEMORY
# ------------------------------------------------------------------------------
IMG_HEIGHT = 640  # Reduced from 1088 (half size)
IMG_WIDTH = 640  # Reduced from 1920 (half size)
NUM_CLASSES = 3
BATCH_SIZE = 3   # Reduced from 2
LEARNING_RATE = 0.001
EPOCHS = 100
PATIENCE = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset paths
TRAIN_IMAGES_DIR = 'CWD-Github/train/images'
TRAIN_MASKS_DIR = 'CWD-Github/train/Morphed_Images'
VAL_IMAGES_DIR = 'CWD-Github/valid/images'
VAL_MASKS_DIR = 'CWD-Github/valid/Morphed_Images'

# ------------------------------------------------------------------------------
# Clear GPU cache at start
# ------------------------------------------------------------------------------
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# ------------------------------------------------------------------------------
# Dataset Class (NO torchvision)
# ------------------------------------------------------------------------------
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.img_height = img_height
        self.img_width = img_width
        self.images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = img_name.replace('.jpg', '_morphed.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Load and resize using PIL
        image = Image.open(img_path).convert('RGB')
        image = image.resize((self.img_width, self.img_height), Image.BILINEAR)

        mask = Image.open(mask_path).convert('L')
        mask = mask.resize((self.img_width, self.img_height), Image.NEAREST)

        # Convert to tensors
        image_array = np.array(image, dtype=np.float32) / 255.0
        image_tensor = torch.from_numpy(image_array).permute(2, 0, 1)  # HWC -> CHW
        mask_tensor = torch.from_numpy(np.array(mask, dtype=np.int64))

        return image_tensor, mask_tensor


# ------------------------------------------------------------------------------
# Create DataLoaders - REDUCED WORKERS
# ------------------------------------------------------------------------------
train_dataset = SegmentationDataset(TRAIN_IMAGES_DIR, TRAIN_MASKS_DIR)
val_dataset = SegmentationDataset(VAL_IMAGES_DIR, VAL_MASKS_DIR)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=True)  # Reduced workers
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=2, pin_memory=True)    # Reduced workers

# ------------------------------------------------------------------------------
# IoU Calculation
# ------------------------------------------------------------------------------
def calculate_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float()
        union = (pred_inds | target_inds).sum().float()

        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append((intersection / union).item())

    return ious


# ------------------------------------------------------------------------------
# Training Function - WITH GRADIENT ACCUMULATION
# ------------------------------------------------------------------------------
def train_epoch(model, dataloader, optimizer, criterion, accumulation_steps=4):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_ious = []

    optimizer.zero_grad()

    for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs, masks)
        loss = loss / accumulation_steps  # Normalize loss
        loss.backward()

        # Only update weights every accumulation_steps
        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * images.size(0) * accumulation_steps
        _, predicted = torch.max(outputs, 1)
        total += masks.numel()
        correct += (predicted == masks).sum().item()

        # Calculate IoU
        batch_ious = calculate_iou(predicted, masks, NUM_CLASSES)
        all_ious.append(batch_ious)

        # Clear cache periodically
        if batch_idx % 50 == 0:
            torch.cuda.empty_cache()

    # Update any remaining gradients
    optimizer.step()
    optimizer.zero_grad()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = 100.0 * correct / total

    # Calculate mean IoU
    valid_ious = [[iou for iou in batch if not np.isnan(iou)] for batch in all_ious]
    mean_iou = np.mean([np.mean(batch) for batch in valid_ious if len(batch) > 0])
    iou_loss = 1 - mean_iou

    return epoch_loss, epoch_acc, mean_iou, iou_loss


# ------------------------------------------------------------------------------
# Evaluation Function - WITH MEMORY CLEANUP
# ------------------------------------------------------------------------------
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_ious = []

    with torch.no_grad():
        for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Validation", leave=False)):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            outputs = model(images)
            loss = criterion(outputs, masks)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += masks.numel()
            correct += (predicted == masks).sum().item()

            # Calculate IoU
            batch_ious = calculate_iou(predicted, masks, NUM_CLASSES)
            all_ious.append(batch_ious)

            # Clear cache periodically
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = 100.0 * correct / total

    # Calculate mean IoU
    valid_ious = [[iou for iou in batch if not np.isnan(iou)] for batch in all_ious]
    mean_iou = np.mean([np.mean(batch) for batch in valid_ious if len(batch) > 0])
    iou_loss = 1 - mean_iou

    return epoch_loss, epoch_acc, mean_iou, iou_loss


# ------------------------------------------------------------------------------
# Train and Evaluate Model Function - MEMORY OPTIMIZED
# ------------------------------------------------------------------------------
def train_and_evaluate_model(model_name, model_class, train_loader, val_loader, epochs, patience):
    """
    Train a model, save the best checkpoint, and log metrics to Excel.
    """
    # Clear GPU cache before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 1) Create a directory for this model
    model_dir = model_name
    os.makedirs(model_dir, exist_ok=True)

    num_gpus = torch.cuda.device_count()
    print(f"Using {num_gpus} GPUs...")

    # 2) Instantiate model + move to device
    model = model_class(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)
    # Don't use DataParallel for single GPU
    if num_gpus > 1:
        model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])

    # 3) Define optimizer + loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_model_path = None
    patience_counter = 0
    records = []

    for epoch in range(epochs):
        print(f"\n[{model_name}] Epoch {epoch+1}/{epochs}")

        train_loss, train_accuracy, train_iou, train_iou_loss = train_epoch(
            model, train_loader, optimizer, criterion, accumulation_steps=4
        )
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Train IoU: {train_iou:.4f}, Train IoU Loss: {train_iou_loss:.4f}")

        val_loss, val_accuracy, val_iou, val_iou_loss = evaluate(model, val_loader, criterion)
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, "
              f"Val IoU: {val_iou:.4f}, Val IoU Loss: {val_iou_loss:.4f}")

        records.append([
            epoch+1,
            train_loss,
            train_accuracy,
            val_loss,
            val_accuracy,
            train_iou_loss,
            train_iou,
            val_iou_loss,
            val_iou
        ])

        # early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = os.path.join(model_dir, "unet_best_model.pth")
            torch.save(model.state_dict(), best_model_path)  # Save state_dict instead of full model
            print(f"  [*] Best model saved at {best_model_path}")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  [!] Early stopping for {model_name}")
                break

        # Clear cache after each epoch
        torch.cuda.empty_cache()

    # 4) Save Training_Metrics.xlsx in model_dir
    excel_path = os.path.join(model_dir, "Training_Metrics.xlsx")
    columns = [
        "Epoch",
        "Training Loss",
        "Training Accuracy",
        "Validation Loss",
        "Validation Accuracy",
        "Training IoU loss",
        "Mean Training IoU",
        "Validation IoU loss",
        "Mean Validation IoU"
    ]
    df = pd.DataFrame(records, columns=columns)
    df.to_excel(excel_path, index=False)
    print(f"  Metrics saved to {excel_path}")

    print(f"Done training {model_name}.\n")

    # Clear memory after training
    del model
    torch.cuda.empty_cache()


# ------------------------------------------------------------------------------
# Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # List of models you want to train
    models_to_train = {
        "DeepLabV3Plus": DeepLabV3Plus,
        "PSPNet": PSPNet,
        "FPN": FPN,
        "UNetPlusPlus": UNetPlusPlus,
        "AttentionUNet": AttentionUNet,
        # "Unet": UNet,
        # "Unet-BN": UNet_BN,
        # "Unet-SA": UNet_SA,
        # "Unet-DSC": UNet_DSC,
        # "Unet-BN-SA-DSC": UNet_SA_BN_DSC
    }

    for model_name, model_class in models_to_train.items():
        train_and_evaluate_model(model_name, model_class,
                                 train_loader, val_loader,
                                 epochs=EPOCHS, patience=PATIENCE)


Using 1 GPUs...

[DeepLabV3Plus] Epoch 1/100


Train Loss: 0.3061, Train Accuracy: 88.07%, Train IoU: 0.7376, Train IoU Loss: 0.2624


Val Loss: 0.1718, Val Accuracy: 93.09%, Val IoU: 0.8436, Val IoU Loss: 0.1564
  [*] Best model saved at DeepLabV3Plus/unet_best_model.pth

[DeepLabV3Plus] Epoch 2/100


Train Loss: 0.1836, Train Accuracy: 92.90%, Train IoU: 0.8182, Train IoU Loss: 0.1818


Val Loss: 0.1586, Val Accuracy: 93.89%, Val IoU: 0.8565, Val IoU Loss: 0.1435
  [*] Best model saved at DeepLabV3Plus/unet_best_model.pth

[DeepLabV3Plus] Epoch 3/100


...

Train Loss: 0.0872, Train Accuracy: 96.40%, Train IoU: 0.9090, Train IoU Loss: 0.0910


Val Loss: 0.0949, Val Accuracy: 96.19%, Val IoU: 0.9086, Val IoU Loss: 0.0914

[DeepLabV3Plus] Epoch 40/100


Train Loss: 0.0842, Train Accuracy: 96.52%, Train IoU: 0.9126, Train IoU Loss: 0.0874


Val Loss: 0.0923, Val Accuracy: 96.42%, Val IoU: 0.9133, Val IoU Loss: 0.0867

[DeepLabV3Plus] Epoch 41/100


Training:  92%|█████████▏| 328/355 [05:15<00:25,  1.05it/s]

In [ ]:
import torch
import gc

# Clear cache and garbage
gc.collect()
torch.cuda.empty_cache()

# If using multiple devices:
torch.cuda.ipc_collect()

# Optionally, print how much memory is free now
print(f"Memory Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Memory Cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")


In [ ]:
#  ml
#     num_gpus = torch.cuda.device_count()
#     print(f"Using {num_gpus} GPUs...")

#     # 2) Instantiate model + move to device
#     model = model_class(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)
#     if num_gpus > 1:
#         model = nn.DataParallel(model,device_ids=[0, 1, 2, 3])

#     # 3) Define optimizer + loss
#     optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
#     criterion = nn.CrossEntropyLoss()

#     best_val_loss   = float('inf')
#     best_model_path = None
#     patience_counter= 0
#     records         = []

#     for epoch in range(epochs):
#         print(f"\n[{model_name}] Epoch {epoch+1}/{epochs}")

#         train_loss, train_accuracy, train_iou, train_iou_loss = train_epoch(model, train_loader, optimizer, criterion)
#         print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
#               f"Train IoU: {train_iou:.4f}, Train IoU Loss: {train_iou_loss:.4f}")

#         val_loss, val_accuracy, val_iou, val_iou_loss = evaluate(model, val_loader, criterion)
#         print(f"Val Loss:   {val_loss:.4f}, Val Accuracy:   {val_accuracy:.2f}%, "
#               f"Val IoU:   {val_iou:.4f}, Val IoU Loss:   {val_iou_loss:.4f}")

#         records.append([
#             epoch+1,
#             train_loss,
#             train_accuracy,
#             val_loss,
#             val_accuracy,
#             train_iou_loss,
#             train_iou,
#             val_iou_loss,
#             val_iou
#         ])

#         # early stopping
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             best_model_path = os.path.join(model_dir, "unet_best_model.pth")
#             torch.save(model, best_model_path)
#             print(f"  [*] Best model saved at {best_model_path}")
#             patience_counter = 0
#         else:
#             patience_counter += 1

#         if patience_counter >= patience:
#             print(f"  [!] Early stopping for {model_name}")
#             break

#     # 4) Save Training_Metrics.xlsx in model_dir
#     excel_path = os.path.join(model_dir, "Training_Metrics.xlsx")
#     columns = [
#         "Epoch",
#         "Training Loss",
#         "Training Accuracy",
#         "Validation Loss",
#         "Validation Accuracy",
#         "Training IoU loss",
#         "Mean Training IoU",
#         "Validation IoU loss",
#         "Mean Validation IoU"
#     ]
#     df = pd.DataFrame(records, columns=columns)
#     df.to_excel(excel_path, index=False)
#     print(f"  Metrics saved to {excel_path}")

#     print(f"Done training {model_name}.\n")


# if __name__ == "__main__":
#     # List of models you want to train
#     models_to_train = {
#         "Unet"         : UNet,
#         "Unet-BN"      : UNet_BN,
#         "Unet-SA"      : UNet_SA,
#         "Unet-DSC"     : UNet_DSC,
#         "Unet-BN-SA-DSC": UNet_SA_BN_DSC
#     }

#     for model_name, model_class in models_to_train.items():
#         train_and_evaluate_model(model_name, model_class,
#                                  train_loader, val_loader,
#                                  epochs=EPOCHS, patience=PATIENCE)




In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

# ======================
# CONFIG
# ======================
TEST_IMAGES_DIR = 'CWD-Github/test/images'
TEST_MASKS_DIR  = 'CWD-Github/test/Morphed_Images'
IMG_HEIGHT      = 1088
IMG_WIDTH       = 1920
NUM_CLASSES     = 3
DEVICE          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ======================
# TRANSFORMS
# ======================
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ======================
# MODEL LOADING
# ======================
def load_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(DEVICE)
    model.eval()
    return model

# ======================
# PREPROCESSING
# ======================
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(DEVICE)

def preprocess_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)

# ======================
# PREDICTION
# ======================
def generate_predictions(model, image):
    with torch.no_grad():
        output = model(image)
        pred = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()

# ======================
# METRICS
# ======================
def calculate_iou(pred_mask, gt_mask, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        iou = intersection / union if union > 0 else 0
        iou_per_class.append(iou)
    return iou_per_class

def calculate_dice(pred_mask, gt_mask, num_classes):
    dice_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        denom = (np.sum(pred_mask == cls) + np.sum(gt_mask == cls))
        dice = 2.0 * intersection / denom if denom > 0 else 0
        dice_per_class.append(dice)
    return dice_per_class

def calculate_jaccard(pred_mask, gt_mask, num_classes):
    jaccard_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        jaccard = intersection / union if union > 0 else 0
        jaccard_per_class.append(jaccard)
    return jaccard_per_class

# ======================
# CONFUSION MATRIX
# ======================
def plot_confusion_matrix(cm, class_names, save_path, title="Confusion Matrix", fmt='d'):
    fig, ax = plt.subplots(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    ax.set_title(title)
    plt.tight_layout()
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)

# ======================
# EVALUATION
# ======================
def evaluate_model(model, test_images_dir, test_masks_dir, num_classes, results_dir):
    all_pred, all_gt = [], []
    iou_per_class = np.zeros(num_classes)
    dice_per_class = np.zeros(num_classes)
    jaccard_per_class = np.zeros(num_classes)
    total_samples = 0

    image_files = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg')]
    for img_name in image_files:
        image_path = os.path.join(test_images_dir, img_name)
        mask_path = os.path.join(test_masks_dir, img_name.replace('.jpg', '_morphed.png'))

        image = preprocess_image(image_path)
        gt_mask = preprocess_mask(mask_path)
        pred_mask = generate_predictions(model, image)

        # Flatten for confusion metrics
        all_pred.extend(pred_mask.flatten())
        all_gt.extend(gt_mask.flatten())

        # Per-class metrics
        iou_sample = calculate_iou(pred_mask, gt_mask, num_classes)
        dice_sample = calculate_dice(pred_mask, gt_mask, num_classes)
        jaccard_sample = calculate_jaccard(pred_mask, gt_mask, num_classes)

        iou_per_class += np.array(iou_sample)
        dice_per_class += np.array(dice_sample)
        jaccard_per_class += np.array(jaccard_sample)
        total_samples += 1

    # Average across samples
    iou_per_class /= total_samples
    dice_per_class /= total_samples
    jaccard_per_class /= total_samples

    mean_iou = np.mean(iou_per_class)
    mean_dice = np.mean(dice_per_class)
    mean_jaccard = np.mean(jaccard_per_class)

    print(f"Mean IoU: {mean_iou:.4f}")
    print(f"Mean Jaccard (mIoU): {mean_jaccard:.4f}")
    print(f"Mean Dice: {mean_dice:.4f}")

    return mean_iou, mean_jaccard, mean_dice

# ======================
# MAIN
# ======================
if __name__ == "__main__":
    model_path = "DeepLabV3Plus/unet_best_model.pth"  # change for PSPNet or others
    model = load_model(model_path)

    mean_iou, mean_jaccard, mean_dice = evaluate_model(
        model=model,
        test_images_dir=TEST_IMAGES_DIR,
        test_masks_dir=TEST_MASKS_DIR,
        num_classes=NUM_CLASSES,
        results_dir=os.path.dirname(model_path)
    )


# Saving Prediction Images Of Each Model

In [ ]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')  # Turn off interactive backend (no pop-up windows)

TEST_IMAGE_FOLDER       = 'CWD-Github/test/images'
GROUND_TRUTH_MASK_FOLDER= 'CWD-Github/test/Morphed_Images'
IMG_HEIGHT              = 1088
IMG_WIDTH               = 1920
NUM_CLASSES             = 3
DEVICE                  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])


def load_full_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image


def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)


def generate_segmentation_mask(model, image):
    with torch.no_grad():
        output = model(image)        # (B, NUM_CLASSES, H, W)
        pred   = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()


def visualize_and_save_comparison(
    model, input_image, gt_mask, class_rgb_mapping, input_image_path,
    save_folder, save_predictions=True
):
    # Load the original image for visualization
    original_image = Image.open(input_image_path).convert('RGB')
    original_image = original_image.resize((IMG_WIDTH, IMG_HEIGHT))

    # Generate predicted mask
    pred_mask = generate_segmentation_mask(model, input_image)

    # Map predicted mask to RGB
    rgb_pred_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
    for class_id, rgb_value in class_rgb_mapping.items():
        rgb_pred_mask[pred_mask == class_id] = rgb_value

    # Map ground truth to RGB
    rgb_gt_mask = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 3), dtype=np.uint8)
    for class_id, rgb_value in class_rgb_mapping.items():
        rgb_gt_mask[gt_mask == class_id] = rgb_value

    # Plot side-by-side
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(original_image)
    axes[0].set_title('Original')
    axes[0].axis('off')

    axes[1].imshow(rgb_gt_mask)
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')

    axes[2].imshow(rgb_pred_mask)
    axes[2].set_title('Predicted')
    axes[2].axis('off')

    fig.tight_layout()

    # Save the figure (no plt.show())
    if save_predictions:
        # For the figure
        fig_filename = os.path.splitext(os.path.basename(input_image_path))[0] + '_compare.png'
        fig_save_path= os.path.join(save_folder, fig_filename)
        fig.savefig(fig_save_path, bbox_inches='tight')

        # For the predicted mask alone
        pred_mask_img = Image.fromarray(rgb_pred_mask)
        pred_filename = os.path.splitext(os.path.basename(input_image_path))[0] + '_predmask.png'
        pred_save_path= os.path.join(save_folder, pred_filename)
        pred_mask_img.save(pred_save_path)

    plt.close(fig)

# -------------------------------------------------------------------
# Class-to-RGB mapping
# -------------------------------------------------------------------
class_rgb_mapping = {
    0: (0, 0, 0),    # black
    1: (0, 255, 0),  # green
    2: (255, 0, 0),  # red
}

# -------------------------------------------------------------------
# MAIN:
# 1) Find all "Unet-..." directories
# 2) For each, load "unet_best_model.pth"
# 3) Randomly pick 5 test images, generate predictions
# 4) Save side-by-side figure + predicted mask
# -------------------------------------------------------------------
if __name__ == "__main__":
    # Silence any console printing
    # (Here we can reassign print to a no-op if needed)
    def no_op(*args, **kwargs):
        pass
    print = no_op

    # 1) Find directories that start with "Unet-"
    all_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

    # 2) For each directory, load unet_best_model.pth
    # and do random predictions
    test_image_files = [
        f for f in os.listdir(TEST_IMAGE_FOLDER)
        if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')
    ]
    # If <5 images exist, use them all
    if len(test_image_files) <= 5:
        selected_images = test_image_files
    else:
        selected_images = random.sample(test_image_files, 5)

    for model_dir in all_dirs:
        model_path = os.path.join(model_dir, "unet_best_model.pth")
        if not os.path.isfile(model_path):
            continue  # skip if no best model in that dir

        # Create a subfolder "Predictions" inside model_dir
        pred_save_folder = os.path.join(model_dir, "Predictions")
        os.makedirs(pred_save_folder, exist_ok=True)

        # Load model
        model = load_full_model(model_path)

        # For each selected image, compare
        for image_file in selected_images:
            input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
            # ground truth
            gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
            gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)

            input_image = preprocess_image(input_image_path)
            ground_truth_mask = load_ground_truth_mask(gt_mask_path)

            # Visualize and save
            visualize_and_save_comparison(
                model=model,
                input_image=input_image,
                gt_mask=ground_truth_mask,
                class_rgb_mapping=class_rgb_mapping,
                input_image_path=input_image_path,
                save_folder=pred_save_folder,
                save_predictions=True
            )


# Evaluating Each Model

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm
from PIL import Image
import time

from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib
matplotlib.use('Agg')  # Turn off interactive display
import matplotlib.pyplot as plt
import seaborn as sns


TEST_IMAGES_DIR = 'CWD-Github/test/images'
TEST_MASKS_DIR  = 'CWD-Github/test/Morphed_Images'
IMG_HEIGHT      = 1088
IMG_WIDTH       = 1920
NUM_CLASSES     = 3
DEVICE          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Suppress console output
def no_op(*args, **kwargs):
    pass
print = no_op  # Overwrite default print

transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

'''
def load_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model.to(DEVICE)
    model.eval()
    return model
'''
def load_model(model_path):
    model = torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu")

    # ✅ Wrap model with DataParallel to use multiple GPUs
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model,device_ids=[0, 1, 2, 3])

    model.to("cuda")  # Move to GPU
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(DEVICE)

def preprocess_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)


def generate_predictions(model, image):
    with torch.no_grad():
        output = model(image)            # [B, NUM_CLASSES, H, W]
        pred   = torch.argmax(output, 1) # pick class with max logit
        return pred.squeeze().cpu().numpy()


def calculate_iou(pred_mask, gt_mask, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union        = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        iou = intersection / union if union > 0 else 0
        iou_per_class.append(iou)
    return iou_per_class

def calculate_dice(pred_mask, gt_mask, num_classes):
    dice_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        denom        = (np.sum(pred_mask == cls) + np.sum(gt_mask == cls))
        dice         = 2.0 * intersection / denom if denom > 0 else 0
        dice_per_class.append(dice)
    return dice_per_class

def calculate_jaccard(pred_mask, gt_mask, num_classes):
    jaccard_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union        = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        jaccard      = intersection / union if union > 0 else 0
        jaccard_per_class.append(jaccard)
    return jaccard_per_class


def plot_confusion_matrix(cm, class_names, save_path, title="Confusion Matrix", fmt='d'):
    """
    'fmt': 'd' for counts, '.2f' for percentages
    Saves the figure to 'save_path'.
    """
    fig, ax = plt.subplots(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    ax.set_title(title)
    plt.tight_layout()
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)


def evaluate_model(model, test_images_dir, test_masks_dir, num_classes, results_dir):
    all_pred = []
    all_gt   = []

    iou_per_class       = np.zeros(num_classes)
    dice_per_class      = np.zeros(num_classes)
    jaccard_per_class   = np.zeros(num_classes)
    accuracy_per_class  = np.zeros(num_classes)
    precision_per_class = np.zeros(num_classes)
    recall_per_class    = np.zeros(num_classes)
    f1_per_class        = np.zeros(num_classes)

    image_files  = [img for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
    total_samples= 0

    for img_name in image_files:
        image_path = os.path.join(test_images_dir, img_name)
        mask_name  = img_name.replace('.jpg', '_morphed.png')
        mask_path  = os.path.join(test_masks_dir, mask_name)

        image   = preprocess_image(image_path)
        gt_mask = preprocess_mask2*(mask_path)
        pred_mask= generate_predictions(model, image)

        # Flatten
        all_pred.extend(pred_mask.flatten())
        all_gt.extend(gt_mask.flatten())

        # iou/dice/jaccard         iou_sample     = calculate_iou(pred_mask,     gt_mask, num_classes)
        dice_sample    = calculate_dice(pred_mask,    gt_mask, num_classes)
        jaccard_sample = calculate_jaccard(pred_mask, gt_mask, num_classes)
        iou_per_class     += np.array(iou_sample)
        dice_per_class    += np.array(dice_sample)
        jaccard_per_class += np.array(jaccard_sample)

        # per-class metrics
        for cls in range(num_classes):
            tp = np.sum((pred_mask == cls) & (gt_mask == cls))
            fp = np.sum((pred_mask == cls) & (gt_mask != cls))
            fn = np.sum((pred_mask != cls) & (gt_mask == cls))
            total_class_pixels= np.sum(gt_mask == cls)
            accuracy_per_class[cls] += tp / (total_class_pixels + 1e-6)

            precision_per_class[cls]+= tp / (tp + fp + 1e-6)
            recall_per_class[cls]   += tp / (tp + fn + 1e-6)
            f1_per_class[cls]       += 2*tp / (2*tp + fp + fn + 1e-6)

        total_samples += 1

    # Normalize
    accuracy_per_class  /= total_samples
    precision_per_class /= total_samples
    recall_per_class    /= total_samples
    f1_per_class        /= total_samples
    iou_per_class       /= total_samples
    dice_per_class      /= total_samples
    jaccard_per_class   /= total_samples

    mean_dice     = np.mean(dice_per_class)
    mean_jaccard  = np.mean(jaccard_per_class)

    # freq weighted iou
    all_gt_arr = np.array(all_gt)
    class_counts= np.bincount(all_gt_arr, minlength=num_classes)
    frequency_weighted_iou= np.average(iou_per_class, weights=class_counts)

    # overall metrics
    all_pred_arr = np.array(all_pred)
    accuracy = (all_pred_arr == all_gt_arr).sum()/len(all_gt_arr)
    precision= precision_score(all_gt_arr, all_pred_arr, average='weighted', zero_division=1)
    recall   = recall_score(all_gt_arr,    all_pred_arr, average='weighted', zero_division=1)
    f1       = f1_score(all_gt_arr,        all_pred_arr, average='weighted', zero_division=1)

    mean_iou    = np.mean(iou_per_class)
    weighted_iou= np.average(iou_per_class, weights=np.bincount(all_gt_arr))

    # confusion matrix
    confusion_matrix_counts= np.zeros((num_classes, num_classes), dtype=np.int64)
    for gt_val, pr_val in zip(all_gt_arr, all_pred_arr):
        confusion_matrix_counts[gt_val, pr_val]+=1

    confusion_matrix_percent= np.zeros_like(confusion_matrix_counts, dtype=float)
    for r in range(num_classes):
        row_sum = confusion_matrix_counts[r,:].sum()
        if row_sum > 0:
            confusion_matrix_percent[r,:] = (confusion_matrix_counts[r,:]/ row_sum)*100

    class_names= [f"Class {i}" for i in range(num_classes)]

    # Save confusion matrices as PNG
    cm_counts_path  = os.path.join(results_dir, "Confusion_Matrix_Counts.png")
    cm_percent_path = os.path.join(results_dir, "Confusion_Matrix_Percent.png")
    plot_confusion_matrix(confusion_matrix_counts,  class_names, save_path=cm_counts_path,  title="Confusion Matrix (Counts)", fmt='d')
    plot_confusion_matrix(confusion_matrix_percent, class_names, save_path=cm_percent_path, title="Confusion Matrix (Percent)", fmt='.2f')

    return (
        accuracy, accuracy_per_class, precision, precision_per_class,
        recall, recall_per_class, f1, f1_per_class,
        iou_per_class, mean_iou, weighted_iou, frequency_weighted_iou,
        dice_per_class, jaccard_per_class, mean_dice, mean_jaccard
    )

def save_results_to_excel(
    model_name,
    accuracy, accuracy_per_class,
    precision, precision_per_class,
    recall, recall_per_class,
    f1, f1_per_class,
    iou_per_class, mean_iou,
    weighted_iou, frequency_weighted_iou,
    dice_per_class, jaccard_per_class,
    mean_dice, mean_jaccard,
    save_directory
):
    # We'll store the final xlsx in the same directory as the model
    excel_path = os.path.join(save_directory, "Performance_Evaluation_Metrics.xlsx")

    columns = (
        ['Model Name', 'Accuracy']
        + [f'Accuracy Class {i}' for i in range(len(accuracy_per_class))]
        + ['Precision'] + [f'Precision Class {i}' for i in range(len(precision_per_class))]
        + ['Recall'] + [f'Recall Class {i}' for i in range(len(recall_per_class))]
        + ['F1 Score'] + [f'F1 Score Class {i}' for i in range(len(f1_per_class))]
        + [f'IoU Class {i}' for i in range(len(iou_per_class))]
        + ['Mean IoU', 'Weighted IoU', 'Frequency Weighted IoU']
        + [f'Dice Coefficient Class {i}' for i in range(len(dice_per_class))]
        + ['Mean Dice']
        + [f'Jaccard Index Class {i}' for i in range(len(jaccard_per_class))]
        + ['Mean Jaccard']
    )

    new_row = {
        'Model Name': model_name,
        'Accuracy': accuracy,
        **{f'Accuracy Class {i}': acc for i, acc in enumerate(accuracy_per_class)},
        'Precision': precision,
        **{f'Precision Class {i}': prec for i, prec in enumerate(precision_per_class)},
        'Recall': recall,
        **{f'Recall Class {i}': r for i, r in enumerate(recall_per_class)},
        'F1 Score': f1,
        **{f'F1 Score Class {i}': f1c for i, f1c in enumerate(f1_per_class)},
        **{f'IoU Class {i}': iou for i, iou in enumerate(iou_per_class)},
        'Mean IoU': mean_iou,
        'Weighted IoU': weighted_iou,
        'Frequency Weighted IoU': frequency_weighted_iou,
        **{f'Dice Coefficient Class {i}': d for i, d in enumerate(dice_per_class)},
        'Mean Dice': mean_dice,
        **{f'Jaccard Index Class {i}': j for i, j in enumerate(jaccard_per_class)},
        'Mean Jaccard': mean_jaccard,
    }

    new_data = pd.DataFrame([new_row], columns=columns)

    # Overwrite any existing file to store only the last row
    new_data.to_excel(excel_path, index=False, header=True)

if __name__ == "__main__":
    # Suppress console output
    def no_op(*args, **kwargs):
        pass
    print = no_op  # Overwrite default print

    # 1) Find all directories that match "Unet-<backbone>"
    all_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

    for model_dir in all_dirs:
        # 2) Load "unet_best_model.pth" in that directory, if exists
        model_path = os.path.join(model_dir, "unet_best_model.pth")
        if not os.path.isfile(model_path):
            continue  # skip if no best model found

        model = load_model(model_path)

        # 3) Evaluate
        results = evaluate_model(
            model=model,
            test_images_dir=TEST_IMAGES_DIR,
            test_masks_dir=TEST_MASKS_DIR,
            num_classes=NUM_CLASSES,
            results_dir=model_dir  # store confusion matrix PNG in same directory
        )

        # 4) Unpack
        (
            accuracy, accuracy_per_class,
            precision, precision_per_class,
            recall, recall_per_class,
            f1, f1_per_class,
            iou_per_class, mean_iou,
            weighted_iou, frequency_weighted_iou,
            dice_per_class, jaccard_per_class,
            mean_dice, mean_jaccard
        ) = results

        # 5) Save row to "Performance_Evaluation_Metrics.xlsx" in model_dir
        save_results_to_excel(
            model_name=model_dir,
            accuracy=accuracy,
            accuracy_per_class=accuracy_per_class,
            precision=precision,
            precision_per_class=precision_per_class,
            recall=recall,
            recall_per_class=recall_per_class,
            f1=f1,
            f1_per_class=f1_per_class,
            iou_per_class=iou_per_class,
            mean_iou=mean_iou,
            weighted_iou=weighted_iou,
            frequency_weighted_iou=frequency_weighted_iou,
            dice_per_class=dice_per_class,
            jaccard_per_class=jaccard_per_class,
            mean_dice=mean_dice,
            mean_jaccard=mean_jaccard,
            save_directory=model_dir
        )


# Saving Training Curves Of Each Model

In [ ]:
import os
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # So figures don't pop up; they are just saved
import matplotlib.pyplot as plt

def plot_training_curves_for_model(excel_path, output_dir):
    """
    Reads Training_Metrics.xlsx from excel_path and saves four plots in output_dir.
    """
    # Read Excel
    df = pd.read_excel(excel_path)

    # Extract columns
    epochs            = df['Epoch']
    train_loss        = df['Training Loss']
    val_loss          = df['Validation Loss']
    train_acc         = df['Training Accuracy']
    val_acc           = df['Validation Accuracy']
    train_iou_loss    = df['Training IoU loss']
    val_iou_loss      = df['Validation IoU loss']
    mean_train_iou    = df['Mean Training IoU']
    mean_val_iou      = df['Mean Validation IoU']

    # 1) Training vs Validation Loss
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_loss, label='Training Loss', marker='o')
    plt.plot(epochs, val_loss,   label='Validation Loss', marker='s')
    plt.title('Training vs Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    loss_plot_path = os.path.join(output_dir, 'Train_vs_Val_Loss.png')
    plt.savefig(loss_plot_path, bbox_inches='tight')
    plt.close()

    # 2) Training vs Validation Accuracy
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_acc, label='Training Accuracy', marker='o')
    plt.plot(epochs, val_acc,   label='Validation Accuracy', marker='s')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

    acc_plot_path = os.path.join(output_dir, 'Train_vs_Val_Accuracy.png')
    plt.savefig(acc_plot_path, bbox_inches='tight')
    plt.close()

    # 3) Training IoU Loss vs Validation IoU Loss
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_iou_loss, label='Training IoU Loss', marker='o')
    plt.plot(epochs, val_iou_loss,   label='Validation IoU Loss', marker='s')
    plt.title('Training vs Validation IoU Loss')
    plt.xlabel('Epoch')
    plt.ylabel('IoU Loss')
    plt.legend()
    plt.grid(True)

    iou_loss_plot_path = os.path.join(output_dir, 'Train_vs_Val_IoU_Loss.png')
    plt.savefig(iou_loss_plot_path, bbox_inches='tight')
    plt.close()

    # 4) Mean Training IoU vs Mean Validation IoU
    plt.figure(figsize=(8,6))
    plt.plot(epochs, mean_train_iou, label='Mean Training IoU', marker='o')
    plt.plot(epochs, mean_val_iou,   label='Mean Validation IoU', marker='s')
    plt.title('Mean Training IoU vs Mean Validation IoU')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.grid(True)

    iou_plot_path = os.path.join(output_dir, 'Mean_Train_vs_Val_IoU.png')
    plt.savefig(iou_plot_path, bbox_inches='tight')
    plt.close()

def main():
    # 1) Find directories named "Unet-..."
    unet_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith('Unet')]

    for unet_dir in unet_dirs:
        # 2) The path to the Training_Metrics.xlsx
        excel_path = os.path.join(unet_dir, 'Training_Metrics.xlsx')
        if not os.path.isfile(excel_path):
            continue  # skip if no metrics file

        # 3) Create "Training_Curves" subdir
        curves_dir = os.path.join(unet_dir, 'Training_Curves')
        os.makedirs(curves_dir, exist_ok=True)

        # 4) Generate + save plots
        plot_training_curves_for_model(excel_path, curves_dir)

if __name__ == "__main__":
    main()


# All Models Performance Evaluation Sheet

In [ ]:
import os
import pandas as pd

# 1) Automatically discover directories named "Unet-..."
model_directories = [
    d for d in os.listdir('.')
    if os.path.isdir(d) and d.startswith("Unet")
]

# 2) Each directory's Excel file name
excel_filename = "Performance_Evaluation_Metrics.xlsx"

# 3) Where to save the merged file
results_folder = "Results"
os.makedirs(results_folder, exist_ok=True)
merged_excel_path = os.path.join(results_folder, "All_Models_Performance_Evaluation_Metrics.xlsx")

# 4) Create an empty list to hold the last rows from each subdirectory
merged_rows = []

# 5) Loop over each Unet-<backbone> directory
for model_dir in model_directories:
    excel_path = os.path.join(model_dir, excel_filename)

    # Check if the file exists
    if not os.path.isfile(excel_path):
        print(f"Warning: {excel_path} not found. Skipping.")
        continue

    # Read entire Excel file
    df = pd.read_excel(excel_path)

    if df.empty:
        print(f"Warning: {excel_path} is empty. Skipping.")
        continue

    # Get the last (bottom) row
    last_row = df.iloc[[-1]].copy()
    merged_rows.append(last_row)

# 6) If we have rows, concatenate them; otherwise create empty DataFrame
if len(merged_rows) > 0:
    merged_df = pd.concat(merged_rows, ignore_index=True)
else:
    merged_df = pd.DataFrame()

# 7) Overwrite the final Excel file with these rows
merged_df.to_excel(merged_excel_path, index=False)
print(f"Merged file saved to: {merged_excel_path}")



# Saving Predictions For All Models

In [ ]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
all_dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
model_dirs = [d for d in all_dirs if d.startswith("Unet-")]
models_info = [(d, os.path.join(d, "unet_best_model.pth")) for d in model_dirs]

# ------------------------------------------------------------------------------
# 2) Define directories / file paths and hyperparameters
# ------------------------------------------------------------------------------
TEST_IMAGE_FOLDER        = 'CWD-Github/test/images/'
GROUND_TRUTH_MASK_FOLDER = 'CWD-Github/test/Morphed_Images/'
PREDICTION_SAVE_FOLDER   = 'Predictions'
os.makedirs(PREDICTION_SAVE_FOLDER, exist_ok=True)

IMG_HEIGHT  = 1088
IMG_WIDTH   = 1920
NUM_CLASSES = 3
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------------------
# 3) Define transformation (as used during training)
# ------------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ------------------------------------------------------------------------------
# 4) Helper functions
# ------------------------------------------------------------------------------
def load_full_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image

def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)

def generate_segmentation_mask(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)  # shape: (B, NUM_CLASSES, H, W)
        pred = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()  # shape: (H, W)

def mask_to_rgb(mask_array):
    h, w = mask_array.shape
    rgb_image = np.zeros((h, w, 3), dtype=np.uint8)
    # Define the mapping for 3 classes
    class_rgb_mapping = {
        0: (0, 0, 0),      # Black for background (or class 0)
        1: (0, 255, 0),    # Green for class 1
        2: (255, 0, 0)     # Red for class 2
    }
    for cls, color in class_rgb_mapping.items():
        rgb_image[mask_array == cls] = color
    return rgb_image

# ------------------------------------------------------------------------------
# 5) Main execution: load models, select images, and create merged figure
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Load all models
    loaded_models = []
    for model_name, model_path in models_info:
        if not os.path.isfile(model_path):
            continue
        model = load_full_model(model_path)
        loaded_models.append((model_name, model))
    if len(loaded_models) == 0:
        exit(0)

    # Gather test images
    test_image_files = [f for f in os.listdir(TEST_IMAGE_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(test_image_files) == 0:
        exit(0)

    # Randomly select up to 5 images
    selected_images = random.sample(test_image_files, 5) if len(test_image_files) >= 5 else test_image_files

    # Set up figure:
    n_rows = len(selected_images)
    n_cols = 2 + len(loaded_models)  # 1: Input, 1: GT, rest: each model's prediction
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 2 * n_rows))
    if n_rows == 1:
        axes = [axes]  # ensure axes is a list of rows

    # Process each image:
    for row_idx, image_file in enumerate(selected_images):
        input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
        gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
        gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)

        # Load and resize input image and ground truth mask for display
        original_image = Image.open(input_image_path).resize((IMG_WIDTH, IMG_HEIGHT))
        gt_mask_np = load_ground_truth_mask(gt_mask_path)
        gt_rgb = mask_to_rgb(gt_mask_np)

        # Preprocess image for model inference
        input_tensor = preprocess_image(input_image_path)

        # Column 0: Input image
        axes[row_idx][0].imshow(original_image)
        if row_idx == 0:
            axes[row_idx][0].set_title("Input Image", fontsize=27)
        axes[row_idx][0].axis('off')

        # Column 1: Ground truth mask
        axes[row_idx][1].imshow(gt_rgb)
        if row_idx == 0:
            axes[row_idx][1].set_title("Ground Truth Mask", fontsize=27)
        axes[row_idx][1].axis('off')

        # Next columns: Predictions from each model
        for model_i, (model_name, model_obj) in enumerate(loaded_models):
            pred_mask = generate_segmentation_mask(model_obj, input_tensor)
            pred_rgb = mask_to_rgb(pred_mask)
            col_idx = 2 + model_i
            axes[row_idx][col_idx].imshow(pred_rgb)
            if row_idx == 0:
                axes[row_idx][col_idx].set_title(f"{model_name} \nPredicted Mask", fontsize=27)
            axes[row_idx][col_idx].axis('off')

    plt.tight_layout()
    merged_filename = os.path.join(PREDICTION_SAVE_FOLDER, "All_Models_Predictions.png")
    plt.savefig(merged_filename, bbox_inches='tight',dpi=200)
    # Uncomment the next line if you wish to display the figure interactively
    plt.show()


# Saving Predictions of Each Model

In [ ]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
model_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

# ------------------------------------------------------------------------------
# 2) Define directories and hyperparameters
# ------------------------------------------------------------------------------
TEST_IMAGE_FOLDER        = 'CWD-Github/test/images/'
GROUND_TRUTH_MASK_FOLDER = 'CWD-Github/test/Morphed_Images/'
# (The merged prediction figure for each model will be saved in a "Predictions" subfolder of that model directory.)
IMG_HEIGHT  = 1088
IMG_WIDTH   = 1920
NUM_CLASSES = 3
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------------------
# 3) Define the transformation (as used during training)
# ------------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ------------------------------------------------------------------------------
# 4) Helper functions
# ------------------------------------------------------------------------------
def load_full_model(model_dir):
    """Load the full model from <model_dir>/unet_best_model.pth."""
    model_path = os.path.join(model_dir, "unet_best_model.pth")
    if not os.path.isfile(model_path):
        return None
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image

def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)

def generate_segmentation_mask(model, image_tensor):
    with torch.no_grad():
        # model outputs logits of shape (B, NUM_CLASSES, H, W)
        output = model(image_tensor)
        pred = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()  # shape: (H, W)

def mask_to_rgb(mask_array):
    h, w = mask_array.shape
    rgb_image = np.zeros((h, w, 3), dtype=np.uint8)
    # Fixed mapping for 3 classes
    class_rgb_mapping = {
        0: (0, 0, 0),      # Black
        1: (0, 255, 0),    # Green
        2: (255, 0, 0)     # Red
    }
    for cls, color in class_rgb_mapping.items():
        rgb_image[mask_array == cls] = color
    return rgb_image

# ------------------------------------------------------------------------------
# 5) Main execution: Process each model directory
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Gather test images (all jpg/jpeg/png)
    test_image_files = [f for f in os.listdir(TEST_IMAGE_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(test_image_files) == 0:
        exit(0)

    # Randomly select up to 5 images
    selected_images = random.sample(test_image_files, 5) if len(test_image_files) >= 5 else test_image_files

    # Process each model directory that starts with "Unet-"
    for model_dir in model_dirs:
        model = load_full_model(model_dir)
        if model is None:
            continue

        # Create a Predictions subfolder inside the model directory
        predictions_dir = os.path.join(model_dir, "Predictions")
        os.makedirs(predictions_dir, exist_ok=True)

        # Set up a figure with one row per image and 3 columns (Input, Ground Truth, Predicted)
        n_rows = len(selected_images)
        n_cols = 3
        fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(4 * n_cols, 2 * n_rows))
        if n_rows == 1:
            axes = [axes]  # Ensure axes is a list of rows

        for row_idx, image_file in enumerate(selected_images):
            input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
            gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
            gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)

            # Load original image and ground truth for display
            original_image = Image.open(input_image_path).resize((IMG_WIDTH, IMG_HEIGHT))
            gt_mask_np = load_ground_truth_mask(gt_mask_path)
            gt_rgb = mask_to_rgb(gt_mask_np)

            # Preprocess image for inference
            input_tensor = preprocess_image(input_image_path)

            # Column 0: Input image
            axes[row_idx][0].imshow(original_image)
            if row_idx == 0:
                axes[row_idx][0].set_title("Input Image", fontsize=27)
            axes[row_idx][0].axis('off')

            # Column 1: Ground truth mask
            axes[row_idx][1].imshow(gt_rgb)
            if row_idx == 0:
                axes[row_idx][1].set_title("Ground Truth Mask", fontsize=27)
            axes[row_idx][1].axis('off')

            # Column 2: Predicted mask from this model
            pred_mask = generate_segmentation_mask(model, input_tensor)
            pred_rgb = mask_to_rgb(pred_mask)
            axes[row_idx][2].imshow(pred_rgb)
            if row_idx == 0:
                axes[row_idx][2].set_title(f"{model_dir} \nPredicted Mask", fontsize=27)
            axes[row_idx][2].axis('off')

        plt.tight_layout()
        merged_filename = os.path.join(predictions_dir, "All_Models_Predictions.png")
        # Save the figure at high resolution
        plt.savefig(merged_filename, bbox_inches='tight', dpi=300)
        plt.close(fig)


# Saving Results of each model

In [ ]:
import os
import pandas as pd

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
model_directories = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

# Name of the Excel file in each model directory
excel_filename = "Performance_Evaluation_Metrics.xlsx"

# Loop over each found model directory
for model_dir in model_directories:
    # Construct the full path to the Excel file
    excel_path = os.path.join(model_dir, excel_filename)

    # Skip this directory if the file does not exist
    if not os.path.isfile(excel_path):
        continue

    # Read the Excel file into a DataFrame
    df = pd.read_excel(excel_path)
    if df.empty:
        continue

    # Get the last (bottom) row from the DataFrame
    last_row = df.iloc[-1]

    # ------------------ Overall Metrics ------------------
    overall_metrics = {
        "Metric": [
            "Accuracy",
            "Precision",
            "Recall",
            "F1 Score",
            "Mean IoU",
            "Frequency Weighted IoU",
            "Mean Dice",
            "Mean Jaccard"
        ],
        "Value": [
            last_row["Accuracy"],
            last_row["Precision"],
            last_row["Recall"],
            last_row["F1 Score"],
            last_row["Mean IoU"],
            last_row["Frequency Weighted IoU"],
            last_row["Mean Dice"],
            last_row["Mean Jaccard"]
        ]
    }
    overall_df = pd.DataFrame(overall_metrics)

    # ------------------ Per-Class Metrics ------------------
    per_class_data = {
        "Class": ["Class 0", "Class 1", "Class 2"],
        "Accuracy": [
            last_row["Accuracy Class 0"],
            last_row["Accuracy Class 1"],
            last_row["Accuracy Class 2"]
        ],
        "Precision": [
            last_row["Precision Class 0"],
            last_row["Precision Class 1"],
            last_row["Precision Class 2"]
        ],
        "Recall": [
            last_row["Recall Class 0"],
            last_row["Recall Class 1"],
            last_row["Recall Class 2"]
        ],
        "F1 Score": [
            last_row["F1 Score Class 0"],
            last_row["F1 Score Class 1"],
            last_row["F1 Score Class 2"]
        ],
        "IoU": [
            last_row["IoU Class 0"],
            last_row["IoU Class 1"],
            last_row["IoU Class 2"]
        ],
        "Dice": [
            last_row["Dice Coefficient Class 0"],
            last_row["Dice Coefficient Class 1"],
            last_row["Dice Coefficient Class 2"]
        ],
        "Jaccard": [
            last_row["Jaccard Index Class 0"],
            last_row["Jaccard Index Class 1"],
            last_row["Jaccard Index Class 2"]
        ]
    }
    per_class_df = pd.DataFrame(per_class_data)

    # ------------------------------------------------------------------------------
    # 3) Save the new Excel files in a Results subfolder of the model directory
    # ------------------------------------------------------------------------------
    results_dir = os.path.join(model_dir, "Results")
    os.makedirs(results_dir, exist_ok=True)

    overall_excel_path = os.path.join(results_dir, "Overall_Metrics.xlsx")
    per_class_excel_path = os.path.join(results_dir, "Per_Class_Metrics.xlsx")

    overall_df.to_excel(overall_excel_path, index=False)
    per_class_df.to_excel(per_class_excel_path, index=False)


# Seperating All Models Perdormance Evaluation Sheet

In [ ]:
import os
import pandas as pd

# ------------------------------------------------------------------------------
# 1) Path to the merged Excel file
# ------------------------------------------------------------------------------
merged_excel_path = os.path.join("Results", "All_Models_Performance_Evaluation_Metrics.xlsx")

# ------------------------------------------------------------------------------
# 2) Read the Excel file into a DataFrame
# ------------------------------------------------------------------------------
df = pd.read_excel(merged_excel_path)

# ------------------------------------------------------------------------------
# 3) Mapping to change model names
# ------------------------------------------------------------------------------
name_mapping = {
    "unet_best_model.pth": "Unet",
    "unetplusplus_best_model.pth": "Unet++",
    "manet_best_model.pth": "MAnet",
    "linknet_best_model.pth": "Linknet",
    "fpn_best_model.pth": "FPN",
    "pspnet_best_model.pth": "PSPNet",
    "pan_best_model.pth": "PAN",
    "deeplabv3_best_model.pth": "DeepLabV3",
    "deeplabv3plus_best_model.pth": "DeepLabV3+",
    "upernet_best_model.pth": "UPerNet",
    "segformer_best_model.pth": "Segformer"
}

# Update the "Model Name" column based on the mapping.
# If a model name is not found in the mapping, leave it unchanged.
df["Model Name"] = df["Model Name"].apply(lambda x: name_mapping.get(x, x))

# ------------------------------------------------------------------------------
# 4) Create Overall Metrics DataFrame
# ------------------------------------------------------------------------------
overall_columns = [
    "Model Name",
    "Accuracy",
    "Precision",
    "Recall",
    "F1 Score",
    "Mean IoU",
    "Weighted IoU",
    "Frequency Weighted IoU",
    "Mean Dice",
    "Mean Jaccard"
]
overall_df = df[overall_columns].copy()

# ------------------------------------------------------------------------------
# 5) Create Per-Class Metrics DataFrame
# ------------------------------------------------------------------------------
per_class_columns = [
    "Model Name",
    "Accuracy Class 0", "Accuracy Class 1", "Accuracy Class 2",
    "Precision Class 0", "Precision Class 1", "Precision Class 2",
    "Recall Class 0", "Recall Class 1", "Recall Class 2",
    "F1 Score Class 0", "F1 Score Class 1", "F1 Score Class 2",
    "IoU Class 0", "IoU Class 1", "IoU Class 2",
    "Dice Coefficient Class 0", "Dice Coefficient Class 1", "Dice Coefficient Class 2",
    "Jaccard Index Class 0", "Jaccard Index Class 1", "Jaccard Index Class 2"
]
per_class_df = df[per_class_columns].copy()

# ------------------------------------------------------------------------------
# 6) Save the two DataFrames to Excel (overwrite if re-executed)
# ------------------------------------------------------------------------------
overall_excel_path = os.path.join("Results", "Overall_Metrics.xlsx")
per_class_excel_path = os.path.join("Results", "Per_Class_Metrics.xlsx")

overall_df.to_excel(overall_excel_path, index=False)
per_class_df.to_excel(per_class_excel_path, index=False)

print(f"Saved overall metrics to: {overall_excel_path}")
print(f"Saved per-class metrics to: {per_class_excel_path}")
